# Twitter Sentiment Analysis for Coinmarketcap

Get the top 10 cryptocurrencies with Coinmarketcap APIs and analyse the user sentiments with Twitter APIs. 

In [1]:
# import libraries for Coinmarketcap
import urllib.request
import json, datetime

In [2]:
# get data from coinmarketcap for each coin
# define the Coinmarketcap API URL for top 10 coins
# https://api.coinmarketcap.com/v2/ticker/?limit=10
url = "https://api.coinmarketcap.com/v2/ticker/?limit=10"

# open Coinmarketcap url
page = urllib.request.urlopen(url)

# read as JSON format the output
data = json.loads(page.read()) # get the API answer as JSON (dictionary)

In [3]:
# create a list with top crypto to search later in Twitter
cryptos = []

# append only the names and the website slug of each crypto
for x, y in (data['data']).items():
    cryptos.append((y['name'], y['website_slug']))
    
print('Top 10 cryptos:')
for crypto in cryptos:
    print(crypto[0],'|' , crypto[1])

Top 10 cryptos:
Bitcoin | bitcoin
Ethereum | ethereum
XRP | ripple
Bitcoin Cash | bitcoin-cash
EOS | eos
Stellar | stellar
Litecoin | litecoin
Cardano | cardano
Tether | tether
Monero | monero


**HACK needed: tweepy will give you an error with some with python 3.7, so you need to repace 'async' variable with let's say 'tr_async' in \Lib\site-packages\streaming.py!** (https://stackoverflow.com/questions/49339502/tweepy-on-macbook-pycharm-async-invalid-syntax)

Thanks to the following projects:
* https://www.youtube.com/watch?v=o_OZdbCzHUA&index=2&list=PL2-dafEMk2A6QKz1mrk1uIGfHkC1zZ6UU
* https://github.com/llSourcell/twitter_sentiment_challenge/blob/master/demo.py
* https://stackabuse.com/accessing-the-twitter-api-with-python/
* https://github.com/shreyans29/thesemicolon/blob/master/livesenti.py
* https://www.youtube.com/watch?v=l9AC98amjSA

Let's use **tweepy** library:

In [4]:
import tweepy
from textblob import TextBlob

Use your Twitter API keys and secrets for consumer and access (https://stackabuse.com/accessing-the-twitter-api-with-python/):

In [5]:
# Enter your keys/secrets as strings in the following fields
credentials = {}  
credentials['CONSUMER_KEY']    = 'YOUR DATA'  
credentials['CONSUMER_SECRET'] = 'YOUR DATA'  
credentials['ACCESS_TOKEN']    = 'YOUR DATA'  
credentials['ACCESS_SECRET']   = 'YOUR DATA'

# Save the credentials object to file
with open("twitter_credentials.json", "w") as file:  
    json.dump(credentials, file)

## One call of tweets

We will process only one time 15 tweets:

In [6]:
# Import the Twython class
from twython import Twython
import re

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:  
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

Retrieve Tweets:

In [9]:
import pandas as pd
import datetime

print('-> Twitter sentiment analysis using only 15 tweets ...')
print(datetime.datetime.today())
# create a dataframe with the mean values
stats = []

# analyse each crypto using one call of tweets
for crypto in cryptos:
    
    # create search with 2 words: crypto name and website_slug
    keywords = " OR ".join(crypto)
    # print('* Tweets for ',keywords)
    
    # Create our query
    query = {'q': keywords,  
            'result_type': 'popular',
            #'count': 15,
            'lang': 'en',
            }

    # create dictionary to retrive tweets info
    dict_ = {'crypto': [], 'date': [], 'text': [], 'polarity': [], 'subjectivity': [], }  
    
    # Search tweets
    for status in python_tweets.search(**query)['statuses']:
        dict_['crypto'].append(keywords)      # type of crypto
        dict_['date'].append(status['created_at']) # date
        dict_['text'].append(status['text'])       # text of tweet
        
        # Perform Sentiment Analysis on Tweets
        # ****************************************
        # correct encoding
        tweet = status['text'].encode("utf-8")
        # use only letters (eliminate strange characters)
        tweet=" ".join(re.findall("[a-zA-Z]+", str(tweet)))
        
        analysis = TextBlob(tweet.strip())
        sentiment = analysis.sentiment
        dict_['polarity'] = sentiment.polarity         # evaluated polarity
        dict_['subjectivity'] = sentiment.subjectivity # evaluated subjectivity

    # Structure data in a pandas DataFrame for easier manipulation for one coin
    df = pd.DataFrame(dict_)  
    mean_polarity = df.loc[:,"polarity"].mean()
    mean_subjectivity = df.loc[:,"subjectivity"].mean()
    
    # add the mean for each coint
    stats.append((crypto[0]+'-'+crypto[1],float(mean_polarity),float(mean_subjectivity)))

# create a dataframe for the mean values
df_stats = pd.DataFrame(stats, columns=['crypto','polarity','subjectivity'])

# Sort the results using polarity descending
df_stats.sort_values(by='polarity', inplace=True, ascending=False)  

# print means of the evaluations
print(df_stats)

print('\n* Max polarity:')
print(df_stats.loc[df_stats['polarity'].idxmax()])

print('\n* Max subjectivity:')
print(df_stats.loc[df_stats['subjectivity'].idxmax()])

-> Twitter sentiment analysis using only 15 tweets ...
2018-11-03 03:02:40.985836
                      crypto  polarity  subjectivity
4                    EOS-eos  0.500000      0.900000
8              Tether-tether  0.500000      0.900000
6          Litecoin-litecoin  0.137500      0.387500
1          Ethereum-ethereum  0.136364      0.454545
5            Stellar-stellar  0.100000      0.325000
0            Bitcoin-bitcoin  0.000000      1.000000
2                 XRP-ripple  0.000000      1.000000
3  Bitcoin Cash-bitcoin-cash  0.000000      0.000000
9              Monero-monero  0.000000      0.000000
7            Cardano-cardano       NaN           NaN

* Max polarity:
crypto          EOS-eos
polarity            0.5
subjectivity        0.9
Name: 4, dtype: object

* Max subjectivity:
crypto          Bitcoin-bitcoin
polarity                      0
subjectivity                  1
Name: 0, dtype: object


Have fun!

2018@muntisa